In [10]:
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import shapely
from shapely.geometry import Polygon, mapping
from rasterio.features import geometry_mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from atlite.gis import ExclusionContainer, shape_availability
import atlite

In [2]:
luisa = '/home/oskar/shared_input/geodata/onshore/wimby/LUISA_basemap_020321_100m.tif'

In [3]:
def get_bounding():
    
    rectx1 = -12
    rectx2 = 44
    recty1 = 33
    recty2 = 72
    
    polygon = Polygon(
    [
        (rectx1, recty1),
        (rectx1, recty2),
        (rectx2, recty2),
        (rectx2, recty1),
        (rectx1, recty1),
    ]
    )
    
    polygon=shapely.segmentize(polygon, max_segment_length=0.5)
    
    b=gpd.GeoDataFrame(geometry=[polygon],crs="EPSG:4326")
 
    return b.to_crs("EPSG:3035")

In [6]:
aggregated_regions = [
    "AT", "BE", "BG", "CH", "CZ", "DE",
    "DK", "EE", "ES", "FI", "FR", "UK",
    "GR", "HR", "HU", "IE", "IT", "LT",
    "LU", "LV", "NL", "NO", "PL", "PT", 
    "RO", "SE", "SI", "SK",
]

In [7]:
europe = (
    gpd
    .read_file('/home/oskar/shared_input/geodata/onshore/shapes/NUTS_RG_10M_2021_4326.geojson')
    .query("NUTS_ID == @aggregated_regions")
    .set_index(["NUTS_ID"])
    .loc[:,['geometry']]
)

In [15]:
rectx1 = -12
rectx2 = 44
recty1 = 33
recty2 = 72

polygon = Polygon(
    [
        (rectx1, recty1),
        (rectx1, recty2),
        (rectx2, recty2),
        (rectx2, recty1),
        (rectx1, recty1),
    ]
)
europe = gpd.clip(europe, polygon)
europe = europe.to_crs(excluder.crs)

In [20]:
def create_binary_raster(input_raster_path, output_raster_path, categories_of_interest):
    """
    Converts a raster to a binary format (1 for present categories, 0 otherwise), reprojects it,
    and writes it to a specified resolution.
    """
    resolution = 100  # desired resolution in meters

    # Get the bounding box in the destination CRS
    bounding_box_gdf = get_bounding()
    minx, miny, maxx, maxy = bounding_box_gdf.total_bounds

    # Open the existing raster
    with rasterio.open(input_raster_path) as src:
        # Calculate new width and height for the desired resolution
        new_width = int((maxx - minx) / resolution)
        new_height = int((maxy - miny) / resolution)
        
        # Calculate the new transform
        new_transform = from_bounds(minx, miny, maxx, maxy, new_width, new_height)

        # Prepare metadata for the output raster
        kwargs = src.meta.copy()
        kwargs.update({
            'driver': 'GTiff',
            'crs': CRS.from_epsg(3035),
            'transform': new_transform,
            'width': new_width,
            'height': new_height,
            'dtype': 'uint8',  # Use uint8 for the binary representation
            'nodata': 0,
            'compress': 'lzw'
        })

        # Create the output raster file
        with rasterio.open(output_raster_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                data = src.read(i)
                # Create the binary mask for the categories of interest
                binary_mask = np.isin(data, categories_of_interest).astype('uint8')

                # Create geometry mask for bounding box
                mask_shape = [mapping(geom) for geom in bounding_box_gdf.geometry]
                bbox_mask = geometry_mask(
                    geometries=mask_shape,
                    invert=True,
                    transform=src.transform,
                    out_shape=(src.height, src.width),
                    all_touched=True
                )

                # Ensure the binary mask matches the bbox mask
                binary_mask = binary_mask & bbox_mask.astype('uint8')

                # Reproject the binary data to the specified resolution
                reprojected = np.empty(shape=(new_height, new_width), dtype='uint8')
                reproject(
                    source=binary_mask,
                    destination=reprojected,
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=new_transform,
                    dst_crs=CRS.from_epsg(3035),
                    resampling=Resampling.nearest
                )

                # Write the reprojected binary data to the output
                dst.write(reprojected, indexes=i)
                
    print(f"Binary raster created at: {output_raster_path}")

In [21]:
# Example usage
input_raster_path = luisa
output_raster_path = '/home/oskar/shared_input/geodata/onshore/wimby/processed/luisa_ports.tif'
categories_of_interest = [1230] # Port areas

create_binary_raster(input_raster_path, output_raster_path, categories_of_interest)

NameError: name 'from_bounds' is not defined

In [ ]:

minx, miny, maxx, maxy = get_bounding().total_bounds
# Open the existing raster
with rasterio.open(slope) as src:
    # Calculate transform of the new intended output dimensions
    new_transform = Affine.translation(minx, maxy) * Affine.scale(
        (maxx - minx) / src.width,
        (miny - maxy) / src.height)

    # Calculate new width and height
    new_width = int((maxx - minx) / new_transform.a)
    new_height = int((maxy - miny) / -new_transform.e)

    kwargs = src.meta.copy()
    kwargs.update({
        'crs': CRS.from_epsg(3035),
        'transform': new_transform,
        'width': new_width,
        'height': new_height,
        'compress': 'lzw'  # Add LZW compression
    })

    with rasterio.open('/home/oskar/shared_input/geodata/onshore/wimby/processed/euro_slope_40degs.tif', 'w', **kwargs) as dst:
        for i in range(1, src.count + 1):
            reproject(
                source=rasterio.band(src, i),
                destination=rasterio.band(dst, i),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=new_transform,
                dst_crs=CRS.from_epsg(3035),
                resampling=Resampling.nearest)